In [1]:
import pickle
import pandas as pd

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [3]:
year = 2023
month = 3
output_file = f'yellow-{year:04d}-{month:02d}.parquet'
categorical = ['PULocationID', 'DOLocationID']

In [4]:
def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)
print(f"The standard deviation of the predicted duration for this dataset is {y_pred.std(ddof=1)}")

The standard deviation of the predicted duration for this dataset is 6.2474897941993985


In [7]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df['predicted_duration'] = y_pred
df_results = df[['ride_id', 'predicted_duration']]
df_results.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)